uses Pima Indians Diabetes Database
https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

Supervised learning
Binary logistic regression + Neural Network


In [ ]:
# import libraries
import numpy, pandas, matplotlib

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import tensorflow

In [ ]:
# read csv
df = pandas.read_csv("diabetes.csv")
df.head() #prints first 5 rows 


In [ ]:
#graphing to see trends per label
# cols = df.columns.tolist()  
# feature_cols = cols[:-1]    

# for feature in feature_cols:
#     matplotlib.pyplot.hist(
#         df[df["Outcome"] == 1][feature],
#         color='blue',
#         label='Diabetes (1)',
#         alpha=0.7,
#         density=True
#     )
#     matplotlib.pyplot.hist(
#         df[df["Outcome"] == 0][feature],
#         color='red',
#         label='No Diabetes (0)',
#         alpha=0.7,
#         density=True
#     )
#     matplotlib.pyplot.title(feature)
#     matplotlib.pyplot.ylabel("Probability")
#     matplotlib.pyplot.xlabel(feature)
#     matplotlib.pyplot.legend()
#     matplotlib.pyplot.show()

DATA SEPARATION


In [ ]:
# split data into x and y (pregnancies - age: x; outcome: y)
y = df['Outcome']
x = df.drop('Outcome', axis=1)

# separate data to get training, valid and test set
# 70% train, 20% valid, remain=10% test
train, valid, test = numpy.split(df.sample(frac=1), [int(0.7*len(df)), int(0.9*len(df))])

# ensure the features (x) have zero mean and unit variance so each feature contributes equally and having no bias
# Oversampling prevents class imbalances by generating synthetic data - no bias towards majority class
def scale_ds(dataframe, over_sample=False):
    x=dataframe[dataframe.columns[:-1]].values
    y=dataframe[dataframe.columns[-1]].values

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    if over_sample:
        ros= RandomOverSampler()
        x, y = ros.fit_resample(x, y)

    data = numpy.hstack((x, numpy.reshape(y, (-1, 1))))

    return data, x, y

# separate into different datasets
train, x_train, y_train = scale_ds(train, over_sample=True)
valid, x_valid, y_valid = scale_ds(valid, over_sample=False)
test, x_test, y_test = scale_ds(test, over_sample=False)

TRAIN LOGISTIC REGRESSION MODEL AND PREDICT

In [41]:
# build model
log_reg_model = LogisticRegression(max_iter=100, random_state=42)
log_reg_model.fit(x_train, y_train)

#predict
y_train_pred = log_reg_model.predict(x_train)
train_acc = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", round(train_acc, 4))
print("Train Classification Report:\n", classification_report(y_train, y_train_pred))

# Validation dataset 
y_valid_pred = log_reg_model.predict(x_valid)
valid_acc = accuracy_score(y_valid, y_valid_pred)
print("Validation Accuracy:", round(valid_acc, 4))
print("Validation Classification Report:\n", classification_report(y_valid, y_valid_pred))

# Test dataset 
y_test_pred = log_reg_model.predict(x_test)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", round(test_acc, 4))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))

Train Accuracy: 0.7522
Train Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.77      0.76       347
           1       0.76      0.73      0.75       347

    accuracy                           0.75       694
   macro avg       0.75      0.75      0.75       694
weighted avg       0.75      0.75      0.75       694

Validation Accuracy: 0.7338
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.73      0.78        99
           1       0.60      0.75      0.67        55

    accuracy                           0.73       154
   macro avg       0.72      0.74      0.72       154
weighted avg       0.75      0.73      0.74       154

Test Accuracy: 0.6883
Test Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.67      0.75        54
           1       0.49      0.74      0.59        23

    accuracy   

Building a Neural Net using tensorflow

In [ ]:
# visualize loss and accuracy
def plot_loss(history):
    matplotlib.pyplot.plot(history.history['loss'], label='loss')
    matplotlib.pyplot.plot(history.history['val_loss'], label='val_loss')
    matplotlib.pyplot.xlabel('Epoch')
    matplotlib.pyplot.ylabel('Binary Crossentropy')
    matplotlib.pyplot.legend()
    matplotlib.pyplot.grid(True)
    matplotlib.pyplot.show()

def plot_accuracy(history):
    matplotlib.pyplot.plot(history.history['accuracy'], label='accuracy')
    matplotlib.pyplot.plot(history.history['val_accuracy'], label='val_accuracy')
    matplotlib.pyplot.xlabel('Epoch')
    matplotlib.pyplot.ylabel('Accuracy')
    matplotlib.pyplot.legend()
    matplotlib.pyplot.grid(True)
    matplotlib.pyplot.show()

In [ ]:
features = 8
# num_nodes = 17 #2n+1 n=num of features

# build a function which can be called to train different FNN models with different arguments so that the best can be found
def train_model(x_train, y_train, num_nodes, dropout_prob, learning_rate, batch_size, epochs):
    fnn_model = tensorflow.keras.Sequential([
            tensorflow.keras.layers.Input(shape=(features,)),  # hyperparameters, tuple
            tensorflow.keras.layers.Dropout(dropout_prob),
            tensorflow.keras.layers.Dense(num_nodes, activation='relu'),
            tensorflow.keras.layers.Dense(num_nodes, activation='relu', ),
            tensorflow.keras.layers.Dense(1, activation='sigmoid') # can easily round this to (0, or 1)
        ])

    fnn_model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate), 
                      loss='binary_crossentropy',
                    metrics=['accuracy']) #Optimizers will adjust the weight of the model based on loss function hence +accuracy

    history = fnn_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_valid, y_valid), verbose=0)

    return fnn_model, history

In [ ]:
# grid to find best model out of many trained models

least_val_loss = float('inf')
least_loss_model = None
epochs = 100

for num_nodes in [17, 64]:
    for dropout_prob in [0.0, 0.2]:
        for learning_rate in [0.005, 0.001, 0.01]:
            for batch_size in [64, 128]:
                print(f"{num_nodes} nodes, {dropout_prob} dropout, {learning_rate} learning rate, {batch_size} batch size")
                model, history = train_model(
                    x_train, y_train,
                    num_nodes=num_nodes,
                    dropout_prob=dropout_prob,
                    learning_rate=learning_rate,
                    batch_size=batch_size,
                    epochs=epochs
                )

                # plot_loss(history)
                # plot_accuracy(history)

                validation_loss, validation_acc = model.evaluate(x_valid, y_valid, verbose=0)
                print(f" validation loss is {validation_loss}, validation accuracy is {validation_acc}")

                # keep best 
                if validation_loss < least_val_loss:
                    least_val_loss = validation_loss
                    least_loss_model = model


 validation loss is 1.4957128763198853, validation accuracy is 0.7142857313156128
17 nodes, 0.0 dropout, 0.01 learning rate, 128 batch size
 validation loss is 1.0421090126037598, validation accuracy is 0.7337662577629089
17 nodes, 0.2 dropout, 0.005 learning rate, 64 batch size
 validation loss is 0.560004711151123, validation accuracy is 0.7337662577629089
17 nodes, 0.2 dropout, 0.005 learning rate, 128 batch size
 validation loss is 0.5501208901405334, validation accuracy is 0.7402597665786743
17 nodes, 0.2 dropout, 0.001 learning rate, 64 batch size
 validation loss is 0.5135273933410645, validation accuracy is 0.7337662577629089
17 nodes, 0.2 dropout, 0.001 learning rate, 128 batch size
 validation loss is 0.5454038977622986, validation accuracy is 0.7402597665786743
17 nodes, 0.2 dropout, 0.01 learning rate, 64 batch size
 validation loss is 0.5821401476860046, validation accuracy is 0.7662337422370911
17 nodes, 0.2 dropout, 0.01 learning rate, 128 batch size
 validation loss is 

In [ ]:
# final test run with the best model
y_prob = least_loss_model.predict(x_test, verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int).reshape(-1,) # round to 0 or 1. threshold=0.5

print("Final Model Classification Score")
print(classification_report(y_test, y_pred, zero_division=0))

Final Model Classification Report
              precision    recall  f1-score   support

           0       0.89      0.59      0.71        54
           1       0.46      0.83      0.59        23

    accuracy                           0.66        77
   macro avg       0.68      0.71      0.65        77
weighted avg       0.76      0.66      0.68        77

